#### Notebook Overview

This notebook reads in, post-processes, and clips the FEDS `allfires` objects to make them compatible for the stepwise time series analysis used in the paper. 

In [3]:
# ! pip install -e .. -q
import geopandas as gpd
import pandas as pd
import s3fs
from glob import glob
# from fireatlas.postprocess import fill_and_merge_allfires
from shapely.geometry import box

In [4]:
s3 = s3fs.S3FileSystem(anon=False) # connect to s3

In [5]:
western_us_bbox = [-125.698046875, 31.676476158707615,
                   -101.00078125, 49.51429477264348]

# Create a Shapely box geometry from the bounding box coordinates
bbox_geometry = box(*western_us_bbox)

# Create a GeoDataFrame from the box geometry with the original CRS
bbox_gdf = gpd.GeoDataFrame({'geometry': [bbox_geometry]}, crs="EPSG:4326")

# Reproject to 'EPSG:9311' (Native FEDS projection)
bbox_gdf_reprojected = bbox_gdf.to_crs('EPSG:9311')

In [6]:
feds_v3_year_af = gpd.read_parquet(f"s3://maap-ops-workspace/shared/gsfc_landslides/FEDSoutput-v3/ArchiveCONUS{2012}/{2012}/allfires_{2012}1231_PM.parq")

In [ ]:
years = [2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021]
for year in years:
    print(f'Starting for {year}')
    feds_v3_year_af = gpd.read_parquet(f"s3://maap-ops-workspace/shared/gsfc_landslides/FEDSoutput-v3/ArchiveCONUS{year}/{year}/allfires_{year}1231_PM.parq")

    # run postprocessing function
    feds_v3_year_af_processed = fill_and_merge_allfires(feds_v3_year_af, (year,12,31,"PM"), client=None, outpath=None)
    print(f'Processed {year}')    

    # clip to western us
    feds_v3_year_af_processed_clipped = feds_v3_year_af_processed.clip(bbox_gdf_reprojected)
    print(f'Clipped {year}')

    # save
    feds_v3_year_af_processed_clipped.to_parquet(f'~/feds_western_us_{year}_af_postprocessed.parquet',index=True)
    print(f'Saved {year}')

    # also load in, clip, and save pixel data
    feds_v3_year_pix = pd.read_csv(f's3://maap-ops-workspace/shared/gsfc_landslides/FEDSoutput-v3/ArchiveCONUS{year}/{year}/allpixels_{year}1231_PM.csv')    
    feds_v3_year_pix['geometry'] = gpd.points_from_xy(feds_v3_year_pix['Lon'], feds_v3_year_pix['Lat'])
    feds_v3_year_pix = gpd.GeoDataFrame(feds_v3_year_pix, geometry='geometry',crs='EPSG:4326')
    feds_v3_year_pix_clipped = feds_v3_year_pix.clip(bbox_gdf) # regular lat lon here
    feds_v3_year_pix_clipped.to_csv(f'~/feds_western_us_{year}_pixels.csv',index=False)
    print(f'Pixel data clipped and saved for {year}\n')

The following notebook `Orland_etal_DataGenerationV2` shows how these files are read in and processed to make the analysis ready data. 